Pegar dataset e requisitos

In [0]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Q9j1a83CuKzsHCGaNulSkNxBm7Dkn7Ln' -O assin2-train-only.xml
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1J3FpQaHxpM-FDfBUyooh-sZF-B-bM_lU' -O assin2-test.xml
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1kb7xq6Mb3eaqe9cOAo70BaG9ypwkIqEU' -O assin2-dev.xml
!wget http://nilc.icmc.usp.br/assin/assin.tar.gz
!tar -xzf assin.tar.gz
%mkdir input
%mv *.xml input
%rm -rf assin.tar.gz
%cd input
!pip install xmltodict

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/I.C. Nádia/Datasets/Train/assin


Gerar dicionário para assin1 e assin2

In [0]:
import pandas as pd
import xmltodict
import os
import re

files_xml = [f for f in os.listdir(os.curdir) if 'xml' in f]
names = list()
xmls = list()

for filename in files_xml:
    with open(filename) as f:
        xml = xmltodict.parse(f.read())
        name = re.sub(r'(.*).xml', r'\1', filename)
        xmls.append(xml)
        names.append(name)

xml_names = dict(zip(names, xmls))

ExpatError: ignored

Gerar formatação para o mt-dnn

In [0]:
formatting = ['id', 'label','premise', 'hipothesis']
output_names = []
output_files = []
for name in names:
    place = xml_names[name]['entailment-corpus']['pair'] 

    if 'only' in name:
        name = name.replace('-only', '')

    output_names.append(re.sub(r'(.+)-(.+)', r'\1-rte_\2', name))
    output_names.append(re.sub(r'(.+)-(.+)', r'\1-sts_\2', name))
    rte = list()
    sts = list()

    for idx, item in enumerate(place):
        rte.append((item['@id'],item['@entailment'],item['t'],item['h']))
        sts.append((item['@id'],item['@similarity'],item['t'],item['h']))

    rte_df = pd.DataFrame(rte, index = None, columns = formatting)
    sts_df = pd.DataFrame(sts, index = None, columns = formatting)

    output_files.append(rte_df)
    output_files.append(sts_df)

for idx, output in enumerate(output_files):
    output_name = '{}.tmp'.format(output_names[idx])
    output.to_csv(output_name, sep = '\t', index = False)

Importar avanço

In [0]:
import pandas as pd
import os
import re

table_names = [f for f in os.listdir(os.curdir) if 'tmp' in f]
tables = list()

for name in table_names:
    table = pd.read_csv(name, sep = '\t', names = formatting)
    tables.append(table)

table_dict = dict(zip(table_names, tables))
!rm -f *.xml

Substituir texto original por traduzido

In [0]:
!cp ../../../Translation/assin-dic.json assin-dic.json
import json

with open('assin-dic.json') as json_file:
    dic = json.load(json_file)

for key in table_dict.keys():
    for col in ['premise', 'hipothesis']:
        table_dict[key][col] = table_dict[key][col].map(dic)

    output_name = re.sub(r'(.*).tmp', r'\1.tsv', key)
    table_dict[key].to_csv(output_name, sep = '\t', index = False, header = None)
!rm -f *.tmp
!rm -f *.json

Remover marcas html

In [ ]:
!pip install ftfy
!cp ../ftfy_assin.sh .
!bash ftfy_assin.sh

Remover aspas que está em excesso na linha 1711 do *assin-ptbr-train*

In [0]:
for filepath in ['assin-ptbr-rte_train.tsv', 'assin-ptbr-sts_train.tsv']:
    with open(filepath, 'r') as f:
        corpus = f.read()
        corpus = corpus.replace('"As long as','As long as')

    with open(filepath, 'w') as f:   
        f.write(corpus)